<a href="https://colab.research.google.com/github/seanrjohnson/proteinotes/blob/main/colab_notebooks/PAEView.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PAEView: Color AlphaFold structures based on PAE scores

[According to DeepMind](https://www.deepmind.com/publications/enabling-high-accuracy-protein-structure-prediction-at-the-proteome-scale), PAE (Predicted Aligned Error), "reports AlphaFold’s expected position error at residue x, when the predicted and true structures are aligned on residue y. This is useful for assessing confidence in global features, especially domain packing. For residues x and y drawn from two different domains, a consistently low PAE at (x, y) suggests AlphaFold is confident about the relative domain positions. Consistently high PAE at (x, y) suggests the relative positions of the domains should not be interpreted. The general approach used to produce PAE can be adapted to predict a variety of superposition-based metrics, including TM-score and GDT."


Visualizing the mapping of PAE to structures can help to identify flexible loops and  domains.


### Instructions

Run ColabFold on your proteins of interest, then upload pairs pdb and json files in the upload cell below, and render them in the PAE plot cell.

To add more pdb/json files, the upload cell can be run multiple times. Multiple selections are allowed each time.

### References

This notebook is inspired by and borrows code from earlier works, including:

* https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb
  * [Mirdita M, Schütze K, Moriwaki Y, Heo L, Ovchinnikov S, Steinegger M. ColabFold: Making protein folding accessible to all. *Nature Methods*, 2022](https://www.nature.com/articles/s41592-022-01488-1) 




In [21]:
#@title Install dependencies and reset list of uploaded files
!pip install -q --no-warn-conflicts py3dmol
uploaded_files = dict()

In [ ]:
#@title Upload pdb and json files

from google.colab import files
from pathlib import Path
from warnings import warn
import json

print("Upload PDB and JSON files")
new_files = files.upload()

uploaded_files.update(new_files)

def match_pdb_to_json(files_dict):
  """
    Input:
      dict of {filename: file_size}, containing both pdb files and json files.
    returns a dict mapping pdb file names to corresponding json file names, raises a warning if no json file is found
  """
  pdb_to_json = dict()


  for key in files_dict.keys():
    path = Path(key)
    if path.suffix == ".pdb":
      if path.stem + "_scores.json" in files_dict:
        pdb_to_json[key] = json.loads(files_dict[path.stem + "_scores.json"])["pae"]
      else:
        warn(f"Warning: No json file found for pdb file {key}")
  print(f"{len(pdb_to_json)} pairs of *.pdb and *_score.json files found")
  return pdb_to_json

pdb_to_json = match_pdb_to_json(uploaded_files)

In [ ]:
#@title Show PAE plot, drag slider to change position

from ipywidgets.widgets.widget_int import IntSlider, IntText
from ipywidgets import widgets
from ipywidgets.widgets.widget_float import FloatSlider
from ipywidgets.widgets.widget_selection import Dropdown
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
import py3Dmol
from matplotlib.cm import get_cmap
from matplotlib.colors import to_hex
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

pdb_file_widget = Dropdown(options=list(pdb_to_json.keys()), description='Structure:')
position_widget = IntSlider(min=1, max=10000, step=1, value=400, continuous_update=False, description='Position:')
scale_max_widget = IntText(30, description='Scale Max:')
colorscheme_widget = Dropdown(options=["bwr","magma","viridis"], description='Color Scheme:')

def plot_paes(paes, max_value, color_scheme, Ls=None, dpi=100, fig=True):
  # From Colabfold
  num_models = len(paes)
  if fig: plt.figure(figsize=(3*num_models,2), dpi=dpi)
  for n,pae in enumerate(paes):
    plt.subplot(1,num_models,n+1)
    plt.title(f"PAE")
    Ln = pae.shape[0]
    plt.imshow(pae,cmap=color_scheme,vmin=0,vmax=max_value,extent=(0, Ln, Ln, 0))
    # if Ls is not None and len(Ls) > 1: plot_ticks(Ls)
    plt.colorbar()
  return plt

def draw_colorbar(colormap, units, scale_min, scale_max):
  fig, ax = plt.subplots(figsize=(6, 1))
  fig.subplots_adjust(bottom=0.5)
  norm = mpl.colors.Normalize(vmin=scale_min, vmax=scale_max)

  fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=colormap),
            cax=ax, orientation='horizontal', label=units)
  return fig


def update_position_widget(args):
    global pdb_to_json
    position_widget.max = len(pdb_to_json[args["new"]])

pdb_file_widget.observe(update_position_widget, 'value')

update_position_widget({"new": pdb_file_widget.value}, ) # initialize the position

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)

def pae_plot_viewer(**args):
  plot_paes([np.array(pdb_to_json[args['pdb_file']])], args['scale_max'], args['color_scheme'])


def draw_3d_viewer(**args):
  global pdb_to_json
  global uploaded_files
  global view
  pae_cmap = get_cmap(args["color_scheme"])
  
  view.removeAllModels()
  view.addModel(uploaded_files[args["pdb_file"]].decode("utf-8"), 'pdb')
  

  PAE_position = args["position"] - 1 

  if (PAE_position < 0):
    PAE_position = 0
  if PAE_position > len(pdb_to_json[args['pdb_file']])-1:
    PAE_position = len(pdb_to_json[args['pdb_file']])-1

  PAE_row = pdb_to_json[args['pdb_file']][PAE_position]
  for r in range(len(PAE_row)):
    view.setStyle({'resi':f"{r+1}"},{'cartoon':{'color':to_hex(pae_cmap(PAE_row[r] / args["scale_max"]))}})
    view.setHoverable({'resi':f"{r+1}"}, True,
            '''function(atom,viewer,event,container){if(!atom.label){atom.label=viewer.addLabel(" ''' + str(round(PAE_row[r],3)) +''' "+atom.resn+":"+atom.resi,{position:atom,backgroundColor:'mintcream',fontColor:'black'});}}''',
            '''function(atom,viewer){if(atom.label){viewer.removeLabel(atom.label);delete atom.label;}}''')

  
  view.zoomTo()
  # draw_colorbar(pae_cmap, "PAE", 0, args["scale_max"]) # for some reason this breaks the updates
  view.show()
  

   
pae_structure_view = widgets.interactive_output(draw_3d_viewer, {'pdb_file': pdb_file_widget, 'position':position_widget, 'scale_max':scale_max_widget, 'color_scheme':colorscheme_widget})
pae_plot_view = widgets.interactive_output(pae_plot_viewer,{'pdb_file': pdb_file_widget, 'scale_max':scale_max_widget, 'color_scheme':colorscheme_widget})
widgets.HBox([widgets.VBox([pdb_file_widget, position_widget, scale_max_widget, colorscheme_widget, pae_plot_view]), pae_structure_view])
    